1) ask for water network  
2) load network and plot topology (color: demands)  
3) ask to select points  
4) show slider to change demands on selected points --> node indices  
5) buttons: new selection + finish demand manipulation  
6) optimize pump speeds with RL and NM  
7) plot topology (color: heads)

In [ ]:
from __future__ import print_function
import os
import ipywidgets as widgets
from ipywidgets import interact
import ipython_blocking

import plotly.graph_objects as go
from epynet import Network

In [ ]:
def load_wds(button):
    button.disabled = True
    if button.value == 'Anytown':
        fpath = os.path.join('..', 'water_networks', 'anytown_master.inp')
    elif button.value == "D-Town":
        fpath = os.path.join('..', 'water_networks', 'd-town_master.inp')
    else:
        print('No definition file found for {}.'.format(button.value))
        raise
    return Network(fpath)

In [ ]:
def assemble_junc_coordinates(wds):
    junc_x = []
    junc_y = []
    junc_z = []
    for junc in wds.junctions:
        junc_x.append(junc.coordinates[0])
        junc_y.append(junc.coordinates[1])
        junc_z.append(junc.elevation)
    return junc_x, junc_y, junc_z

In [ ]:
def assemble_pipe_coords(wds):
    pipe_x = []
    pipe_y = []
    pipe_z = []
    for pipe in wds.pipes:
        if (pipe.from_node.index in list(wds.junctions.index)) and (pipe.to_node.index in list(wds.junctions.index)):
            pipe_x.append(pipe.from_node.coordinates[0])
            pipe_x.append(pipe.to_node.coordinates[0])
            pipe_x.append(float('nan'))

            pipe_y.append(pipe.from_node.coordinates[1])
            pipe_y.append(pipe.to_node.coordinates[1])
            pipe_y.append(float('nan'))

            pipe_z.append(pipe.from_node.elevation)
            pipe_z.append(pipe.to_node.elevation)
            pipe_z.append(float('nan'))
    return pipe_x, pipe_y, pipe_z

In [ ]:
def build_2d_traces(junc_coords, pipe_coords):
    junc_trace = go.Scatter(
        x = junc_coords[0],
        y = junc_coords[1],
        mode='markers'
        )
    
    pipe_trace = go.Scatter(
        x = pipe_coords[0],
        y = pipe_coords[1],
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )
    return junc_trace, pipe_trace

In [ ]:
def demand_plot(junc_trace, pipe_trace, wds):
    junc_trace.marker = dict(
    showscale = True,
    colorscale = 'YlGnBu',
    reversescale = True,
    color = list(wds.junctions.basedemand),
    size = 10,
    colorbar = dict(
        thickness = 15,
        title = 'Water consumption, m**3/h',
        xanchor = 'left',
        titleside = 'right'
        )
    )

    fig = go.Figure(
    data = [pipe_trace, junc_trace],
    layout = go.Layout(
        #title = wds_btn.value,
        titlefont_size = 16,
        showlegend = False,
        hovermode = 'closest',
        margin = dict(b=20,l=5,r=5,t=40),
        xaxis = dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis = dict(showgrid=False, zeroline=False, showticklabels=False)
        )
    )
    return fig

In [ ]:
#wds = load_wds(btn_select_wds)
wds = Network(os.path.join('..', 'water_networks', 'anytown_master.inp'))
orig_demands = wds.junctions.basedemand
wds.solve()

In [ ]:
junc_coords = assemble_junc_coordinates(wds)
pipe_coords = assemble_pipe_coords(wds)
junc_trace, pipe_trace = build_2d_traces(junc_coords, pipe_coords)
fig = demand_plot(junc_trace, pipe_trace, wds)
fig.show()

In [ ]:
btn_select_wds = widgets.RadioButtons(
    options = ["Anytown", "D-Town"],
    value = "Anytown",
    disabled = False)

btn_load_wds = widgets.Button(
    description = "Load"
)

In [ ]:
btn_select_wds

In [ ]:
btn_load_wds

In [ ]:
%blockrun btn_load_wds

In [ ]:
junc_trace.marker = dict(
    showscale = True,
    colorscale = 'YlGnBu',
    reversescale = True,
    color = list(wds.junctions.head),
    size = 10,
    colorbar = dict(
        thickness = 15,
        title = 'Head, m',
        xanchor = 'left',
        titleside = 'right'
    ))

fig = go.FigureWidget(
    data = [pipe_trace, junc_trace],
    layout = go.Layout(
        #title = wds_btn.value,
        titlefont_size = 16,
        showlegend = False,
        hovermode = 'closest',
        margin = dict(b=20,l=5,r=5,t=40),
        xaxis = dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis = dict(showgrid=False, zeroline=False, showticklabels=False))
        )

In [ ]:
w = widgets.IntSlider(value=75)
w

In [ ]:
button = widgets.Button(description="Click Me!")

display(button)

def on_button_clicked(b):
    global fig
    for pump in wds.pumps:
        pump.speed = w.value / 100 + .3
    wds.solve()
    fig.update_traces(marker = dict(color=list(wds.junctions.head)))
    fig.plotly_update()

button.on_click(on_button_clicked)
display(widgets.VBox([fig]))